<a href="https://colab.research.google.com/github/drkulkarni236/w266_finalproject/blob/main/M_BERT_With_Classification_Cap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers

In [4]:
import numpy as np

import os

import pandas as pd

import tensorflow as tf
import tensorflow_datasets as tfds

import transformers

from transformers import BertTokenizer, TFBertModel
from tensorflow.keras import backend as K

import logging
tf.get_logger().setLevel(logging.ERROR)

In [5]:
tf.config.list_physical_devices('TPU')

[]

In [6]:
tf.__version__

'2.8.0'

In [7]:
transformers.__version__

'4.17.0'

In [8]:
#For saving models
!pip install pyyaml h5py

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [9]:
#Get Data

#Korean Train
from google.colab import drive
drive.mount('/content/gdrive')

korean_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Data_Korean/train.tsv',
                       sep = '\t', 
                       dtype={'comments':str,'hate':str},
                       nrows = 7897)

print("Dataframe Number of (rows, columns) : {}".format(korean_df.shape))

korean_train_df = korean_df[['comments','hate']][:6317]
korean_dev_df = korean_df[['comments','hate']][6317:]


pd.options.display.max_colwidth = 200
korean_train_df.head(10)

Mounted at /content/gdrive
Dataframe Number of (rows, columns) : (7896, 4)


,comments,hate
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네....,hate
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을 미처 알지못했네요ㅠ,none
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래야, 공정한 사회지...심은대로 거두거라...",hate
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",none
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각이 없노,hate
5,10+8 진짜 이승기랑 비교된다,none
6,"100년안에 남녀간 성전쟁 한번 크게 치룬 후 일부다처제, 여성의 정치참여 금지, 여성 투표권 삭제가 세계의 공통문화로 자리잡을듯. 암탉이 너무 울어댐.",hate
7,10년뒤 윤서인은 분명히 재평가될것임. 말하나하나가 틀린게없음,none
8,10년만에 재미를 느끼는 프로였는데왜 니들때문에 폐지를해야되냐,offensive
9,10년차방탄팬인데 우리방탄처럼 성공은못하겠지만 일단 방탄의 부하가되고싶다는거니 이름기억은해둠ㅇㅇ,offensive


In [10]:
korean_dev_df.head(10)

,comments,hate
6317,"장애인이라는 큰 벽을 가지고 자기목표를 정진하고 열심히 산다는게 쉬운일은 아닌데.. 부럽기도 하고, 본 받아야 할 점이네요삼가 고인의 명복을 빕니다",none
6318,장위안한테 카메라 들이대고 물어보면 재밌겠다. 만약에 중국 욕하면 장위안도 인체신비전에서 언제나 볼 수 있을텐데 ㅎㅎ,hate
6319,장윤정 결혼잘했지..가수나부랭이주제에,offensive
6320,장윤정 그냥 재수없는타입 여자가 억세고 드세보인다 도경한 깨갱거리며살것같다,hate
6321,장윤정 남동생이 축구했으니 너도 축구선수할꺼야.,offensive
6322,장윤정 노래는 들으면 감정이입되고 같이 눈물 나고 그런데 얜 아주 감정이 메말라서 무슨 기계가 노래하는 것같고 결정적으로 노래를 ㅈ 못부른다 무슨 수로 1위했는지 모르겠다 또 투표로 조작했나,hate
6323,"장윤정 만나서 사람됐지~~ 오죽하면 장윤정이 결혼조건으로 프리선언금지, kbs퇴사금지 걸었겠냐?? 얘도 한때는 많이 놀았다든데~~",hate
6324,장윤정 좋아요~ 난 저 특유의 콧소리가 좋음. 계속 흥하길~~,none
6325,장윤정부부 돈도많다며 구지 예능 모하러나와 몬난이남매 자랑하려 그러는갑네 거만한 장윤정 티비서 보기실다,offensive
6326,장윤정이 담주 전설이 웬말이냐...막장으로가네...그리고 판정단 입장시킬때 귀지확인하고 입장시켜라...막귀들만있나...박기영 빼고 다 그냥 쑈다,hate


In [11]:
#Get Korean Test Data
korean_test_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Data_Korean/dev.tsv',
                       sep = '\t', 
                       dtype={'comments':str,'hate':str},
                       nrows = 7897)

print("Dataframe Number of (rows, columns) : {}".format(korean_test_df.shape))

korean_test_df = korean_test_df[['comments','hate']]

pd.options.display.max_colwidth = 200
korean_test_df.head(10)

Dataframe Number of (rows, columns) : (471, 4)


,comments,hate
0,송중기 시대극은 믿고본다. 첫회 신선하고 좋았다.,none
1,지현우 나쁜놈,offensive
2,알바쓰고많이만들면되지 돈욕심없으면골목식당왜나온겨 기댕기게나하고 산에가서팔어라,hate
3,설마 ㅈ 현정 작가 아니지??,hate
4,이미자씨 송혜교씨 돈이 그리 많으면 탈세말고 그돈으로 평소에 불우이웃에게 기부도 좀 하고사시죠.,offensive
5,일베충들 ㅂㄷ거리는것봐라 ㅉㅉ,hate
6,아이즈원 힘내세요...일본 진출도 했으니 일본서 좋은 모습 보여줘도 팬들은 응원 합니다.,none
7,강부자 선생님 전미선 비보에 오열을 하셨다니 눈물이 나네요 힘내세요,none
8,알았어 그만,offensive
9,이영자씨는 진정성 있는거라면 녹화불참으로 끝내지말고 자진하차해라 시청자는 고려도 안하고 일방적 불참은 아닌듯 엠비씨도 시청율 좋아서 고민하는거 같은데 결방할게 아니고 폐지해라,offensive


In [12]:
#Replace hate and offensive with 1, none with 0 for Korean

korean_train_df['hate'] = korean_train_df['hate'].replace(['hate'], 1)
korean_train_df['hate'] = korean_train_df['hate'].replace(['offensive'], 1)
korean_train_df['hate'] = korean_train_df['hate'].replace(['none'], 0)

korean_dev_df['hate'] = korean_dev_df['hate'].replace(['hate'], 1)
korean_dev_df['hate'] = korean_dev_df['hate'].replace(['offensive'], 1)
korean_dev_df['hate'] = korean_dev_df['hate'].replace(['none'], 0)

korean_test_df['hate'] = korean_test_df['hate'].replace(['hate'], 1)
korean_test_df['hate'] = korean_test_df['hate'].replace(['offensive'], 1)
korean_test_df['hate'] = korean_test_df['hate'].replace(['none'], 0)

In [13]:
#Get hindi train data, split into train and dev
from google.colab import drive
drive.mount('/content/gdrive')

hindi_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Data_Hindi/hindi_dataset.tsv',
                       sep = '\t', 
                       dtype={'comments':str,'hate':str},
                       nrows = 7897)

print("Dataframe Number of (rows, columns) : {}".format(hindi_df.shape))

hindi_train_df = hindi_df[['text','task_1']][:3732]
hindi_dev_df = hindi_df[['text','task_1']][3732:]

pd.options.display.max_colwidth = 200
hindi_train_df.head(10)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Dataframe Number of (rows, columns) : (4665, 5)


,text,task_1
0,"बांग्लादेश की शानदार वापसी, भारत को 314 रन पर रोका #INDvBAN #CWC19",NOT
1,सब रंडी नाच देखने मे व्यस्त जैसे ही कोई #शांतीदूत के साथ कुछ होगा सब #रंडीरोना शुरू कर देंगे,HOF
2,तुम जैसे हरामियों के लिए बस जूतों की कमी है शुक्र कर अभी तुम्हारी लिंचिंग हुई नहीं है हिंदुओं के जागने की देर है सच में होगी अभी तो तुम जैसे हरामी सुवर ड्रामा बनाएं हो सुवर कहीं का मौलाना। तुम...,HOF
3,"बीजेपी MLA आकाश विजयवर्गीय जेल से रिहा, जमानत मिलने की खुशी में एक समर्थक ने इंदौर में हर्ष फायरिंग की #AkashVijayvargiya https://abpnews.abplive.in/india-news/celebratory-firing-outside-bjp-ml...",NOT
4,"चमकी बुखार: विधानसभा परिसर में आरजेडी का प्रदर्शन, तेजस्वी यादव नदारद #biharencephalitisdeaths https://abpnews.abplive.in/bihar-news/aes-deaths-rjd-protest-in-vidhan-sabha-campus-but-tejashw...",NOT
5,मुंबई में बारिश से लोगों को काफी समस्या हो रही है,NOT
6,Ahmed's dad:-- beta aaj teri mammy kyu nahi baat kr rhi h. Ahmed....,NOT
7,"5 लाख मुसलमान उर्स में, अजमेर की दरगाह पर आते हैं, सिर्फ 300 पुलिस वालों के भरोसे, 2 लाख हिंदुओं की अमरनाथ यात्रा के लिए, 80 हजार कमांडो, पैरामिलिट्री फोर्स, तथा करोड़ों के उपकरण लगाए जा रहे है...",NOT
8,"Do mahashaktiyan mili hain, charo taraf khusi ki leher hai, khus hone wale khus hi rhe hain aur bhakton ko taklif ho rhi hai, khair honi bhi chahiye.",NOT
9,Chants of 'Jai Sri Ram' as Owaisi takes oath: AIMIM chief responds with 'Jai Bhim' @thenewsminute @asadowaisi,NOT


In [14]:
hindi_dev_df

,text,task_1
3732,तेलंगाना में वन विभाग के अधिकारियों के साथ मारपीट का मामला आया सामने @Ashi_IndiaToday,NOT
3733,"मोदी जी का मक़सद है जहां चुनाव हो या होने वाला हो सिर्फ़ वही पे दिखावा किया जाए, रोया जाय रैली में चिल्लाया जाय। बाकी जहां चुनाव नही उसकी तरफ़ मोदी जी देखना भी नही चाहते बच्चे मरे, किसान मरे चा...",HOF
3734,बृजेश ठाकुर हिंदू के 34 अनाथ बेटियों का रेप करके जमीन में दफना दिया दलाल मीडिया की हिम्मत नहीं हुई कि उससे सवाल पूछ ले हिंदुस्तान का मीडिया मुर्दाबाद गरीबों की आवाज अन्याय के खिलाफ लड़ने वाले ...,HOF
3735,नीच को नीच ही दिखते हैं। तू #हिन्दुस्तान का सबसे बड़ा #नीच है। जो एक #सुअर का प्रचार करने के लिए गया था। जिताया नहीं अपने #सुअर_बाप को नीच,HOF
3736,Salute sister...shaandaar jindabaad# jai hind # थोड़ा देसी हो जाऊ तो क्या फर्क# पढ़ा लिखा इंसान दिल से चोट खा सकता है पर # जिन्होंने पैदा किया बड़ा किया निभाया#उससे कभी नहीं#हमेशा अपनो की इज्जत क...,NOT
...,...,...
4660,पाकिस्तान ने हिंदुओं के ख़िलाफ़ बोलने वाले को बर्खास्त किया है । क्या भारत में भी मुसलमानों के ख़िलाफ़ ज़हर उगलने वालों को मोदी जी करेंगे बर्खास्त ? ⁦@narendramodi⁩ ⁦@BJP4India⁩ ⁦@RSSorg⁩ ⁦@INCIndia⁩,NOT
4661,कोहली है #नेहरू नहीं जो अंग्रेजों के तलवे चाटने लगे । भड़वा साला #जवाहर,HOF
4662,परशुराम? वही जिसने अपनी मां की हत्या की थीं?,NOT
4663,जिस देश में #कन्हैया_कुमार जैसा पढ़ा लिखा युवा हार जाए #आतंकवादी जीत जाये उस देश की जनता क्या ?,HOF


In [15]:
#Get Hindi test data
hindi_test_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Data_Hindi/hasoc2019_hi_test_gold_2919.tsv',
                       sep = '\t', 
                       dtype={'comments':str,'hate':str},
                       nrows = 7897)

print("Dataframe Number of (rows, columns) : {}".format(hindi_test_df.shape))

hindi_test_df = hindi_test_df[['text','task_1']]

pd.options.display.max_colwidth = 200
hindi_test_df.head(10)

Dataframe Number of (rows, columns) : (1318, 5)


,text,task_1
0,"वक्त, इन्सान और इंग्लैंड का मौसम आपको कभी भी धोखा दे सकता है। #IndiaVsPakistan",NOT
1,#कांग्रेस के इस #कमीने की #करतूत को देखिए देश की रक्षा करते #शहीद हुए मां भारती के #सपूतों का ख्याल भी नहीं रखा #दुश्मन से गले मिल गये #शर्मनाक. ये पूरे शहीद का अपमान किया है। मैं बहुत कद्र करते ...,HOF
2,पाकिस्तान को फेकना था फेका गया। जो हार कर भी दुश्मन को हरा दे उसी को हमने नाम दिया। @BCCI ये तुम्हारे समझ के बाहर हैं तुम बॉलीवुड में रंडी रोओ इधर मत घुसो।,HOF
3,जो शब्द तूम आज किसी और औरत के लिए यूज कर रहे वो बचाकर रखना क्योंकि यही कल कोई और कल तुम्हारी माँ बहन और बेटी के लिये यूज करेगा तब गुस्सा न करना क्योंकि वो तुम्हारे फेंके शब्द तुम्ही को मारेगा.... ...,NOT
4,नेता जी हम समाजवादी सिपाही हमेशा आपके साथ है आपका हर लिया गया निर्णय हमे स्वीकार है निवेदन है कि कार्यकर्ताओं पे वरिष्ठ नेता ध्यान दे ये चुनाव भी एक सबक है हमारे लिए कही न कही कार्यकर्ताओं को नजरअ...,NOT
5,@narendramodi @AmitShah @BJP4India @BharatKaPM @TeamDeol #AayegaTohModiHi #BJP4India #BJP #कन्हैया_कुमार ने खोदा कुआँ सिंधिया हारे जुआ शत्रुघ्न खामोश हुआ राज बब्बर बना चूहा 22 राज्य ने 00 छुआ खैर ...,HOF
6,"#कुत्ते भी हो रहे हैं #किडनी और #डायबटीज जैसी #बीमारियों का शिकार, जानें वजह&#8230;",NOT
7,VIVAHIT BaitION KO BHI Patrick KIRSI BHOOMI MAIN BAITON K SAMAN U.P MAIN Adhikar KOUN dilanai KI Kirpa karaiga ? Vicharniy pirsn ? Kya vivahit baition k sath anyay hota rahaiga u.p.main ?,HOF
8,10 एजेंसियों को किसी भी कंप्यूटर की निगरानी और डाटा की जांच का अधिकार दिए जाने के खिलाफ दायर याचिकाओं पर सुप्रीम कोर्ट ने सुनवाई टाली। याचिकाओं में सरकार के आदेश को मनमाना बताया गया है। याचिकाकर्त...,HOF
9,"कश्मीर मे सेना पर पत्थर मारने वाले और पत्थरबाजों के हमदर्द, और, लस्सी न देने पर मर्डर करने वाले लोग पूछ रहे हैं कि बाइक चोर को क्यो पीटा?? तबरेज की पत्नी को मुआवजा और नौकरी देगा दिल्ली वक्फ बोर...",NOT


In [16]:
#Convert Hate to 1, Non to 0
hindi_train_df['task_1'] = hindi_train_df['task_1'].replace(['HOF'], 1)
hindi_train_df['task_1'] = hindi_train_df['task_1'].replace(['NOT'], 0)

hindi_dev_df['task_1'] = hindi_dev_df['task_1'].replace(['HOF'], 1)
hindi_dev_df['task_1'] = hindi_dev_df['task_1'].replace(['NOT'], 0)

hindi_test_df['task_1'] = hindi_test_df['task_1'].replace(['HOF'], 1)
hindi_test_df['task_1'] = hindi_test_df['task_1'].replace(['NOT'], 0)

In [17]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
mbert_model = TFBertModel.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [18]:
#Tokenize text
max_length = 512

#Korean
korean_x_train = tokenizer(list(korean_train_df.comments), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
korean_y_train = korean_train_df.hate

korean_x_dev = tokenizer(list(korean_dev_df.comments), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
korean_y_dev = korean_dev_df.hate

korean_x_test = tokenizer(list(korean_test_df.comments), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
korean_y_test = korean_test_df.hate


#Hindi
hindi_x_train = tokenizer(list(hindi_train_df.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
hindi_y_train = hindi_train_df.task_1

hindi_x_dev = tokenizer(list(hindi_dev_df.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
hindi_y_dev = hindi_dev_df.task_1

hindi_x_test = tokenizer(list(hindi_test_df.text), 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
hindi_y_test = hindi_test_df.task_1

In [19]:
def create_classification_model(hidden_sizes = [200], 
                                train_layers = -1, 
                                optimizer=tf.keras.optimizers.Adam(),
                                hidden_initializer = 'he_uniform',
                                dropout = True):
    """
    Build a simple classification model with mBERT. Let's keep it simple and don't add dropout, layer norms, etc.
    """

    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                  'token_type_ids': token_type_ids,
                  'attention_mask': attention_mask}


    #restrict training to the train_layers outer transformer layers
    if not train_layers == -1:

            retrain_layers = []

            for retrain_layer_number in range(train_layers):

                layer_code = '_' + str(11 - retrain_layer_number)
                retrain_layers.append(layer_code)

            for w in mbert_model.weights:
                if not any([x in w.name for x in retrain_layers]):
                    w._trainable = False


    bert_out = mbert_model(bert_inputs)


    classification_token = tf.keras.layers.Lambda(lambda x: x[:,0,:], name='get_first_vector')(bert_out[0])


    hidden = tf.keras.layers.Dense(hidden_sizes[0], name='hidden_layer', activation='relu', kernel_initializer = hidden_initializer)(classification_token)
    hidden = tf.keras.layers.LayerNormalization()(hidden)

    if dropout:
      hidden = tf.keras.layers.Dropout(0.25)(hidden)

    # hidden = tf.keras.layers.Dense(hidden_sizes[1], name='hidden_layer_2', activation='relu')(hidden)
    # hidden = tf.keras.layers.LayerNormalization()(hidden)

    classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden)

    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], 
                                          outputs=[classification])
    
    classification_model.compile(optimizer=optimizer,
                            loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                            metrics='accuracy')


    return classification_model

In [20]:
classification_model = create_classification_model(optimizer=tf.keras.optimizers.Adam(), 
                                                   train_layers = 0,
                                                   hidden_initializer = 'he_uniform',
                                                   dropout = False
                                                   )
classification_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask_layer (InputLay  [(None, 512)]       0           []                               
 er)                                                                                              
                                                                                                  
 input_ids_layer (InputLayer)   [(None, 512)]        0           []                               
                                                                                                  
 token_type_ids_layer (InputLay  [(None, 512)]       0           []                               
 er)                                                                                              
                                                                                              

In [22]:
#Create model with Korean train data and validate on korean dev data



classification_model.fit([korean_x_train.input_ids, korean_x_train.token_type_ids, korean_x_train.attention_mask],
                         korean_y_train,
                         validation_data=([korean_x_dev.input_ids, korean_x_dev.token_type_ids, korean_x_dev.attention_mask],
                         korean_y_dev),
                        epochs=3,
                        batch_size=16)

Epoch 1/3
395/395 [==============================] - 228s 561ms/step - loss: 0.6570 - accuracy: 0.6201 - val_loss: 0.6128 - val_accuracy: 0.6605
Epoch 2/3
395/395 [==============================] - 218s 552ms/step - loss: 0.6235 - accuracy: 0.6443 - val_loss: 0.5937 - val_accuracy: 0.6878
Epoch 3/3
395/395 [==============================] - 218s 552ms/step - loss: 0.6174 - accuracy: 0.6544 - val_loss: 0.5852 - val_accuracy: 0.6909


In [23]:
classification_model.predict([korean_x_train.input_ids, korean_x_train.token_type_ids, korean_x_train.attention_mask], 
                             batch_size=8, 
                             steps=2)

array([[0.63468724],
       [0.64712983],
       [0.7188055 ],
       [0.44873542],
       [0.86339194],
       [0.25892657],
       [0.8369397 ],
       [0.6187344 ],
       [0.73948634],
       [0.6810506 ],
       [0.54799455],
       [0.6692984 ],
       [0.5290894 ],
       [0.6967115 ],
       [0.7662557 ],
       [0.51622343]], dtype=float32)

In [24]:
#Hindi Model
classification_model = create_classification_model(optimizer=tf.keras.optimizers.Adam(), 
                                                   train_layers = 0,
                                                   hidden_initializer = 'he_uniform',
                                                   dropout = False
                                                   )

classification_model.fit([hindi_x_train.input_ids, hindi_x_train.token_type_ids, hindi_x_train.attention_mask],
                         hindi_y_train,
                         validation_data=([hindi_x_dev.input_ids, hindi_x_dev.token_type_ids, hindi_x_dev.attention_mask],
                         hindi_y_dev),
                        epochs=3,
                        batch_size=16)

Epoch 1/3
234/234 [==============================] - 139s 565ms/step - loss: 0.5827 - accuracy: 0.6977 - val_loss: 0.6445 - val_accuracy: 0.6463
Epoch 2/3
234/234 [==============================] - 129s 553ms/step - loss: 0.5264 - accuracy: 0.7412 - val_loss: 0.6991 - val_accuracy: 0.5927
Epoch 3/3
234/234 [==============================] - 129s 550ms/step - loss: 0.5058 - accuracy: 0.7495 - val_loss: 0.9422 - val_accuracy: 0.4920


In [25]:
classification_model = create_classification_model(optimizer=tf.keras.optimizers.Adam(.00005), 
                                                   train_layers = -1,
                                                   hidden_initializer = 'he_uniform',
                                                   dropout = False
                                                   )

classification_model.fit([korean_x_train.input_ids, korean_x_train.token_type_ids, korean_x_train.attention_mask],
                         korean_y_train,
                         validation_data=([korean_x_dev.input_ids, korean_x_dev.token_type_ids, korean_x_dev.attention_mask],
                         korean_y_dev),
                        epochs=3,
                        batch_size=16)

Epoch 1/3
395/395 [==============================] - 229s 560ms/step - loss: 0.7076 - accuracy: 0.5748 - val_loss: 0.6350 - val_accuracy: 0.6447
Epoch 2/3
395/395 [==============================] - 217s 550ms/step - loss: 0.6514 - accuracy: 0.6190 - val_loss: 0.6148 - val_accuracy: 0.6719
Epoch 3/3
395/395 [==============================] - 217s 549ms/step - loss: 0.6312 - accuracy: 0.6481 - val_loss: 0.6094 - val_accuracy: 0.6745


In [26]:
classification_model = create_classification_model(optimizer=tf.keras.optimizers.Adam(.00005), 
                                                   train_layers = -1,
                                                   hidden_initializer = 'he_uniform',
                                                   dropout = False
                                                   )

classification_model.fit([hindi_x_train.input_ids, hindi_x_train.token_type_ids, hindi_x_train.attention_mask],
                         hindi_y_train,
                         validation_data=([hindi_x_dev.input_ids, hindi_x_dev.token_type_ids, hindi_x_dev.attention_mask],
                         hindi_y_dev),
                        epochs=3,
                        batch_size=16)

Epoch 1/3
234/234 [==============================] - 137s 558ms/step - loss: 0.6312 - accuracy: 0.6490 - val_loss: 0.7180 - val_accuracy: 0.5841
Epoch 2/3
234/234 [==============================] - 127s 544ms/step - loss: 0.5676 - accuracy: 0.7053 - val_loss: 0.7178 - val_accuracy: 0.5959
Epoch 3/3
234/234 [==============================] - 127s 545ms/step - loss: 0.5404 - accuracy: 0.7272 - val_loss: 0.6658 - val_accuracy: 0.6217


In [27]:
classification_model = create_classification_model(optimizer=tf.keras.optimizers.Adam(), 
                                                   train_layers = -1,
                                                   hidden_initializer = 'he_uniform',
                                                   dropout = False
                                                   )

classification_model.fit([korean_x_train.input_ids, korean_x_train.token_type_ids, korean_x_train.attention_mask],
                         korean_y_train,
                         validation_data=([korean_x_dev.input_ids, korean_x_dev.token_type_ids, korean_x_dev.attention_mask],
                         korean_y_dev),
                        epochs=3,
                        batch_size=16)

Epoch 1/3
395/395 [==============================] - 226s 554ms/step - loss: 0.6637 - accuracy: 0.6210 - val_loss: 0.5982 - val_accuracy: 0.6865
Epoch 2/3
395/395 [==============================] - 215s 546ms/step - loss: 0.6239 - accuracy: 0.6506 - val_loss: 0.5919 - val_accuracy: 0.6928
Epoch 3/3
395/395 [==============================] - 215s 546ms/step - loss: 0.6249 - accuracy: 0.6505 - val_loss: 0.5909 - val_accuracy: 0.6916


In [28]:
classification_model = create_classification_model(optimizer=tf.keras.optimizers.Adam(), 
                                                   train_layers = -1,
                                                   hidden_initializer = 'he_uniform',
                                                   dropout = False
                                                   )

classification_model.fit([hindi_x_train.input_ids, hindi_x_train.token_type_ids, hindi_x_train.attention_mask],
                         hindi_y_train,
                         validation_data=([hindi_x_dev.input_ids, hindi_x_dev.token_type_ids, hindi_x_dev.attention_mask],
                         hindi_y_dev),
                        epochs=3,
                        batch_size=16)

Epoch 1/3
234/234 [==============================] - 137s 558ms/step - loss: 0.5800 - accuracy: 0.7002 - val_loss: 0.8524 - val_accuracy: 0.4952
Epoch 2/3
234/234 [==============================] - 127s 543ms/step - loss: 0.5213 - accuracy: 0.7345 - val_loss: 0.8339 - val_accuracy: 0.5552
Epoch 3/3
234/234 [==============================] - 127s 543ms/step - loss: 0.5067 - accuracy: 0.7489 - val_loss: 0.8122 - val_accuracy: 0.5273


In [29]:
classification_model = create_classification_model(optimizer=tf.keras.optimizers.Adam(.00005), 
                                                   train_layers = -1,
                                                   hidden_initializer = 'he_uniform',
                                                   dropout = True
                                                   )

classification_model.fit([korean_x_train.input_ids, korean_x_train.token_type_ids, korean_x_train.attention_mask],
                         korean_y_train,
                         validation_data=([korean_x_dev.input_ids, korean_x_dev.token_type_ids, korean_x_dev.attention_mask],
                         korean_y_dev),
                        epochs=3,
                        batch_size=16)

Epoch 1/3
395/395 [==============================] - 225s 553ms/step - loss: 0.7331 - accuracy: 0.5607 - val_loss: 0.6316 - val_accuracy: 0.6453
Epoch 2/3
395/395 [==============================] - 215s 545ms/step - loss: 0.6743 - accuracy: 0.6000 - val_loss: 0.6211 - val_accuracy: 0.6580
Epoch 3/3
395/395 [==============================] - 215s 545ms/step - loss: 0.6564 - accuracy: 0.6144 - val_loss: 0.6123 - val_accuracy: 0.6631


In [30]:
classification_model = create_classification_model(optimizer=tf.keras.optimizers.Adam(.00005), 
                                                   train_layers = -1,
                                                   hidden_initializer = 'he_uniform',
                                                   dropout = True
                                                   )

classification_model.fit([hindi_x_train.input_ids, hindi_x_train.token_type_ids, hindi_x_train.attention_mask],
                         hindi_y_train,
                         validation_data=([hindi_x_dev.input_ids, hindi_x_dev.token_type_ids, hindi_x_dev.attention_mask],
                         hindi_y_dev),
                        epochs=3,
                        batch_size=16)

Epoch 1/3
234/234 [==============================] - 137s 557ms/step - loss: 0.6998 - accuracy: 0.5911 - val_loss: 0.6933 - val_accuracy: 0.6066
Epoch 2/3
234/234 [==============================] - 127s 544ms/step - loss: 0.6322 - accuracy: 0.6605 - val_loss: 0.6852 - val_accuracy: 0.6163
Epoch 3/3
234/234 [==============================] - 127s 544ms/step - loss: 0.5956 - accuracy: 0.6902 - val_loss: 0.6901 - val_accuracy: 0.6174
